In [ ]:
from utils import dataProcess, graph
import torch
import torch.nn as nn
import numpy as np
import dgl
import matplotlib.pyplot as plt

In [ ]:
from random import seed
from tracemalloc import start
import dgl, torch
import numpy as np
import torch
import torch.nn as nn
import scipy.sparse as sp
import tqdm
import argparse
from utils import dataProcess,graph
from model import embed, predict

In [ ]:



def main():
    torch.manual_seed(36)
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset",default="epinions")
    parser.add_argument("--r1",default=0.01,type=float)
    parser.add_argument("--r2",default=0.01,type=float)
    parser.add_argument("--r3",default=0.1,type=float)
    

    args = parser.parse_args(args=[])
    r1 , r2, r3 = args.r1, args.r2, args.r3
    min_test_rmse , min_test_mae = 5.0, 5.0

    # load and clean data from xx_rating.mat and xx_trust.mat file
    dataset = args.dataset
    dataset = "ciao"
    rating_data, trust_data =  dataProcess.get_data_from_matfile(dataset)
    # rating_data, trust_data = dataProcess.load_data_yelp()
    # rating_data = dataProcess.load_data_1m()
    # trust_data = None
    rating_data, trust_data , user_num, item_num = dataProcess.data_clean(rating_data, trust_data)

    rating_data_train, rating_data_valid, rating_data_test = dataProcess.data_split(rating_data,train_ratio=0.8)
    user_info, item_info = dataProcess.stat_info(rating_data_train, user_num,  item_num)
    item_avg ,item_std , user_bias  = dataProcess.get_some_stat(user_info, item_info)
    rating_data_train = dataProcess.add_pref_col(rating_data_train, user_info, item_info)
    trust_data = dataProcess.add_trust_strength(trust_data, rating_data_train)
    

    node_num = user_num + item_num

    
    # generate graph 
    rate_g  = graph.rating_graph_gen(rating_data_train, user_num, item_num)
    trust_g = graph.trust_graph_gen(trust_data, user_num)
    union_g = graph.union_graph_gen(rating_data_train, trust_data, user_num, item_num)
    # g = union_g  # use (rate_g + trust_g) or union_g
    g = rate_g
    # g = union_g
    edge_rate = g.edata['p']
    edge_rate = torch.unsqueeze(edge_rate,0).float().T
    # g = graph.graph_add_aug_r(g, user_info, item_info)

    

    epoch = 60
    batch_size = 512  # each batch use  batch_size users records.
    batch_num = (user_num + batch_size -1) // batch_size 
    hide_dims = 32
    

    device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
    print("device: ", device)
    g = g.to(device)
    trust_g = trust_g.to(device)
    edge_rate = edge_rate.to(device)
    node_onehot_tensor = dataProcess.get_node_onehot_tensor(node_num).to(device)
    node_embed_layer =  embed.NodeEmbed(node_num, hide_dims, g, trust_g).to(device)
    pref_predict_layer = predict.prefPredictLayer(hide_dims).to(device)
    item_avg ,item_std , user_bias  =  torch.tensor(item_avg).float().to(device), torch.tensor(item_std).float().to(device), torch.tensor(user_bias).float().to(device)
    user_group_ratio = torch.tensor([0.1]*user_num).to(device)
    item_avg.requires_grad = True
    item_std.requires_grad = True
    user_group_ratio.requires_grad = True
    user_bias.requires_grad = True

    loss_func = nn.MSELoss()
    r1, r2 ,r3  = 0.01, 0.01, 0.01 # r1 < 0.001, r2 < 0.01
    opt = torch.optim.Adam([
        {'params': node_embed_layer.parameters(), 'weight_decay': 0.001},
        # {'params': node_embed_layer.w.parameters(), 'weight_decay': 0.0001,'lr':0.005}, # < 0.01
        # {'params': node_embed_layer.gat.parameters(), 'weight_decay': r2}, 
        # {'params': node_embed_layer.pref_embeding.parameters(), 'weight_decay': 0.001,'lr':0.0005}, 
        # {'params': node_embed_layer.trust_embeding.parameters(), 'weight_decay': 0.001,'lr':0.0005}, 

        # {'params': node_embed_layer.egat.parameters(), 'weight_decay': 0.01,'lr':0.0001}, 
        # {'params': node_embed_layer.egat2.parameters(), 'weight_decay': 0.01,'lr':0.0001}, 
        # {'params': node_embed_layer.embeds_user_agg.parameters(), 'weight_decay': 0.01}, 
        # {'params': node_embed_layer.embeds_item_agg.parameters(), 'weight_decay': 0.01}, 
        
        # {'params': node_embed_layer.egat2.parameters(), 'weight_decay': r2}, 
        # {'params': node_embed_layer2.gat.parameters(), 'weight_decay': r2}, 
        # {'params': node_embed_layer.egat3.par ameters(), 'weight_decay': r2},

        {'params':pref_predict_layer.mlp.parameters(), 'weight_decay': 0.001, 'lr':0.001}, # lr <0.01
        {'params':user_bias, 'lr':0.01},
        # {'params':item_avg, 'lr':0.001},
        # {'params':item_std, 'lr':0.01},
        {'params':user_group_ratio, 'lr':0.001},
        # {'params':pref_predict_layer.info_mlp2.parameters(), 'weight_decay': 0.04}, # > 0.04
        # {'params':pref_predict_layer.final.parameters(), 'weight_decay': 0.04},
        # {'params':pref_predict_layer.parameters(), 'weight_decay': 0.04},
        # {'params':predict_layer.parameters(), 'weight_decay': r3},
        ],
        lr=0.001)

    np.random.seed(36)
    user_ids_shuffle_list = np.random.permutation(user_num)
    user_rateRecords_dict = dataProcess.get_user_rateRecords_dict(rating_data_train)

    print("===start train===")
    for i in range(epoch):
        epoch_loss = 0
        for j in range(batch_num):
            user_start_idx, user_end_idx  = j * batch_size, min((j+1)*batch_size, user_num)
            batch_user_ids = user_ids_shuffle_list[user_start_idx : user_end_idx]
            batch = dataProcess.get_records(user_rateRecords_dict, batch_user_ids)

            batch_user, batch_item = batch[:,0], batch[:,1]
            batch_rate = torch.tensor(batch[:,2]).float().to(device)
 
            user_embeds,group_embeds, item_embeds  = node_embed_layer(node_onehot_tensor)   # one-hot  ==> user_embed, item_embed
            
            batch_user_embed, batch_group_embeds, batch_item_embed = user_embeds[batch_user], group_embeds[batch_user],item_embeds[batch_item]

            batch_predict_pref_i = pref_predict_layer(batch_user_embed,  batch_item_embed).view(-1)
            batch_predict_pref_g = pref_predict_layer(batch_group_embeds, batch_item_embed).view(-1)
            batch_predict_pref = (1 - user_group_ratio[batch_user]) * batch_predict_pref_i + user_group_ratio[batch_user] * batch_predict_pref_g

            # batch_predict_rate =  item_avg[batch_item] +   item_std[batch_item] * batch_predict_pref + user_bias[batch_user]
            batch_pref =  (batch_rate - user_bias[batch_user] - item_avg[batch_item] )/ item_std[batch_item]

            batch_loss = loss_func(batch_predict_pref *10, batch_pref  * 10)   + torch.abs(torch.sum(user_bias))/torch.sum(torch.abs(user_bias)) 
            # batch_loss = loss_func(batch_predict_pref *10, batch_pref *10 )   
                            
            opt.zero_grad()
            batch_loss.backward()
            opt.step()
            epoch_loss  += batch_loss.item() / batch_num



        with torch.no_grad():
            # data = rating_data_test
            data = rating_data_valid
            test_user_ids, test_item_ids = data[:,0], data[:,1]
            rate = torch.tensor(data[:,2]).float().to(device)

            user_embeds, group_embeds, item_embeds  = node_embed_layer(node_onehot_tensor)
            test_user_embed = user_embeds[test_user_ids]
            test_group_embeds = group_embeds[test_user_ids]
            test_item_embed = item_embeds[test_item_ids]
         
            predict_pref_i= pref_predict_layer(test_user_embed, test_item_embed).view(-1) 
            predict_pref_g= pref_predict_layer(test_group_embeds, test_item_embed).view(-1) 
            predict_pref = (1 - user_group_ratio[test_user_ids]) * predict_pref_i + user_group_ratio[test_user_ids] * predict_pref_g

            predict_rate =  item_avg[test_item_ids] +   item_std[test_item_ids] * predict_pref + user_bias[test_user_ids]
            # predict_rate =  item_avg[test_item_ids]  + user_bias[test_user_ids]
            predict_rate = predict_rate.clamp(1,5)
            # predict_rate = torch.round(predict_rate)

            if i == 15:
                # print(predict_pref[:10])
                # print(item_avg[test_item_ids][:10])
                # print(rate[:10])
                # print(predict_rate[:10])
                print(user_group_ratio[:20])
                print(user_bias[:20])
                # print(node_embed_layer.pref_embeding.weight)
                print(dataProcess.get_recall(predict_rate,rate,4))
                # print( predict_pref[:100])
                return


            test_loss = dataProcess.get_rmse(predict_rate, rate)
            test_mae = dataProcess.get_mae(predict_rate, rate)

        print("epoch :",i + 1,end="\t")
        print("train Loss:", epoch_loss,end="\t")
        print("test RMSE:", test_loss, "     test MAE: ", test_mae)    
        if test_loss < min_test_rmse:
            min_test_rmse = test_loss
            min_test_mae = test_mae
        if i>0 and i % 20 == 0:
            dataProcess.adjust_learning_rate(opt)
    
    with open("utils/log.md","a") as f:
        f.write("# Modify:   \n")
        f.write("   r1 r2, r3: {}, {}, {} \n".format(r1,r2,r3))
        f.write("   Train_Loss:{}   test_MSE:{}    test_mae: {}\n".format( epoch_loss, min_test_rmse, min_test_mae))
        


if __name__ == "__main__":
    main()

